### Задание 1.
#### Давайте посмотрим на продажи авокадо в двух городах (NewYork, LosAngeles) и узнаем, сколько авокадо типа organic было продано в целом к концу каждой недели (накопительная сумма продаж), начиная с начала периода наблюдений (04/01/15)

In [ ]:
select
    region,
    date,
    total_volume,
    sum(total_volume) OVER w as volume
from
    avocado
where 
    region in ('NewYork', 'LosAngeles') AND
    type = 'organic'
window w as
    (
    PARTITION BY region
    order by date
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    )
order by
    region desc,
    date asc

### Задание 2.
#### Посмотрим, когда объемы продаж обычных (conventional) авокадо резко падали по сравнению с предыдущей неделей. Возьмите данные по США в целом, посчитайте разницу между объемом продаж в неделю x (total_volume) и количеством проданных авокадо в течение предыдущей недели. Значения запишите в новый столбец week_diff.

In [ ]:
select
    region,
    date,
    type,
    total_volume,
    total_volume - lag(total_volume, 1) OVER w week_diff
from
    avocado
where 
    region = 'TotalUS' AND
    type = 'conventional'
window w as
    (
    PARTITION BY region
    order by date
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    )
order by
    region desc,
    date asc

### Задание 3.
#### Посмотрим более подробно на объемы продаж авокадо в Нью-Йорке (NewYork) в 2018 году. Создайте колонку с разницей объемов продаж за неделю и за неделю до этого для каждого типа авокадо. Найдите день, когда продажи авокадо типа organic увеличились по сравнению с предыдущей неделей, а conventional – наоборот упали.


In [ ]:
select
    region,
    year,
    total_volume,
    type,
    date,
    total_volume - lag(total_volume, 1) OVER w difference
from
    avocado
where
    region = 'NewYork' and year = '2018'
window w as
    (
    PARTITION BY type
    order by date
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    )
order by
    type

### Задание 4.
#### Теперь посчитайте скользящее среднее цены авокадо (average_price) в Нью-Йорке с разбивкой по типу авокадо. В качестве окна используйте текущую неделю и предыдущие две (обратите внимание, что в данной таблице в строках содержатся данные за неделю, а не за один день).

In [ ]:
select
    date,
    average_price,
    region,
    type,
    avg(average_price) OVER w as rolling_price
from
    avocado
where
    region = 'NewYork'
window w as
    (
    PARTITION BY type
    order by date
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    )